# dateや季節のデータを持ったデータを取得したのでそれに合わせて
# ダミー変換に手を加えたトレインデータ作成

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ

### 前処理

In [2]:
def make_PCA_df(PCA_arr):#PCAで削減したものは二次元配列で帰ってくるので、それをデータフレームにして返す関数
    X=[0]*len(PCA_arr)
    Y=[0]*len(PCA_arr)
    index=0
    for arr in PCA_arr:
        X[index]=arr[0]
        Y[index]=arr[1]
        index+=1
    return pd.DataFrame({'X':X,'Y':Y})

def making_pred_df(df):
    pred_race_df=df
    #pred_race_df=pred_race_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
    result_df=pred_race_df
    result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
    result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22})#新人のave_st_timeを0.22に
    result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02})#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
    result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02})
    #ダミー変数化
    #月のダミー化
    date_dummie_df=pd.get_dummies(result_df['month'])
    months=np.arange(1,13,1)
    for month in months:
        result_df['month_{}'.format(month)]=date_dummie_df[month]
    #季節のダミー化
    date_dummie_df=pd.get_dummies(result_df['season'])
    seasons=['sp','su','au','wi']
    for season in seasons:
        result_df['season_{}'.format(season)]=date_dummie_df[season]
    #開催日数のダミー化
    date_dummie_df=pd.get_dummies(result_df['range_date'])
    ranges=[4,5,6,7]
    for num_range in ranges:
        result_df['range_{}'.format(num_range)]=date_dummie_df[num_range]

    #開催日数のうちの何日目かのダミー化
    date_dummie_df=pd.get_dummies(result_df['num_date'])
    num_dates=[1,2,3,4,5,6,7]
    for num_date in num_dates:
        result_df['num_date_{}'.format(num_date)]=date_dummie_df[num_date]
    result_df=result_df.drop(['month','season','range_date','num_date'],axis=1)
    
    
    result_df_dummie=result_df
    race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
    for column, val in race_dummie_df.iteritems():
        result_df_dummie['race_{}'.format(int(column))]=val
    result_df_dummie=result_df_dummie.drop('number_race',axis=1)

    cols=list(result_df_dummie.columns)
    male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す

    #===========================新規、性別の取り出し機能が良くなかったため作り直す
    empty_arr=[0]*len(result_df_dummie)
    for col in male_cols:
        for number in np.arange(0,2,1):
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        male_dummie_df=pd.get_dummies(result_df_dummie[col])#性別をダミー化
        for column, val in male_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

    cols=list(result_df_dummie.columns)
    moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
    boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す
    #boat もmoterも番号は1~99とする
    numbers=np.arange(1, 100, 1)
    empty_arr=[0]*len(result_df_dummie)
    for col in moter_cols:
        for number in numbers:
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        moter_dummie_df=pd.get_dummies(result_df_dummie[col])#モータ番号をダミー化
        for column, val in moter_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

    #boat番号をダミー化
    for col in boat_cols:
        for number in numbers:
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        boat_dummie_df=pd.get_dummies(result_df_dummie[col])#boat番号をダミー化
        for column, val in boat_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)
    result_df=result_df_dummie



    #クラスタリング
    #分けてみるクラスタの数は[8,10]の2個
    #cluster_target_df　　trainのデータからリザルトと配当金を取り除いたもの
    target_num_cluster=[8,10]
    #test_clustaring_df=train_has_PCA_df
    clustar_target_df=result_df
    clustaring_df=clustar_target_df
    for num_cluster in target_num_cluster:
        pred = KMeans(n_clusters=num_cluster).fit_predict(clustar_target_df)
        clustaring_df['num={}'.format(num_cluster)]=pred

    model_df=clustaring_df

    return model_df

In [4]:
#予測の検証データを2019年に絞り込んで、欠損地の加工を施す
train_df=pd.read_csv('../csv/train_asiya_date.csv',encoding='utf-8')
train_df=train_df.drop(["Unnamed: 0"],axis=1)
test_date_df=train_df[train_df['year']==2019]
"""
test_df=test_date_df.copy()
date_cols=['date','year','month','day','num_date','range_date','season']
date_df=pd.DataFrame(index=test_date_df.index)
for col in date_cols:
    date_df[col]=test_date_df[col]
    test_df=test_df.drop(col,axis=1)
money_col=test_df['money']#配当金情報の削除
test_df=test_df.drop('money',axis=1)
result_col=test_df['result_com']#着の組み合わせ
test_df=test_df.drop('result_com',axis=1)
test_race_df=making_pred_df(test_df)#モデルに突っ込むだけの状態に加工（新人選手の情報、欠損地加工とダミー化）
"""
test_date_df

,date,money,number_race,racer_1_ID,racer_1_age,racer_1_ave_st,racer_1_bo,racer_1_doub,racer_1_male,racer_1_mo,...,racer_6_male,racer_6_mo,racer_6_rank,result_com,year,month,day,num_date,range_date,season
15171,2019-04-01,5780.0,2,4501,31.0,0.16,19.0,0.453,0,72.0,...,0,11.0,2,7,2019,4,1,1,6,sp
15172,2019-04-01,790.0,3,3302,52.0,0.18,17.0,0.328,0,15.0,...,0,23.0,2,5,2019,4,1,1,6,sp
15173,2019-04-01,390.0,4,4050,38.0,0.16,13.0,0.525,0,68.0,...,0,41.0,2,6,2019,4,1,1,6,sp
15174,2019-04-01,3060.0,5,4373,31.0,0.19,51.0,0.344,0,44.0,...,0,20.0,1,31,2019,4,1,1,6,sp
15175,2019-04-01,1360.0,6,4419,32.0,0.20,63.0,0.245,0,50.0,...,0,57.0,4,1,2019,4,1,1,6,sp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,2019-03-25,2190.0,8,3834,45.0,0.16,53.0,0.234,1,43.0,...,1,46.0,3,80,2019,3,25,6,6,sp
17286,2019-03-25,1170.0,9,4319,34.0,0.18,56.0,0.404,1,66.0,...,1,67.0,4,20,2019,3,25,6,6,sp
17287,2019-03-25,840.0,10,4543,32.0,0.19,48.0,0.398,1,14.0,...,1,29.0,4,2,2019,3,25,6,6,sp
17288,2019-03-25,1760.0,11,4448,32.0,0.16,57.0,0.483,1,21.0,...,1,54.0,2,5,2019,3,25,6,6,sp


In [12]:
#月のダミー化
date_dummie_df=pd.get_dummies(test_date_df['month'])
months=np.arange(1,13,1)
for month in months:
    test_date_df['month_{}'.format(month)]=date_dummie_df[month]
#季節のダミー化
date_dummie_df=pd.get_dummies(test_date_df['season'])
seasons=['sp','su','au','wi']
for season in seasons:
    test_date_df['season_{}'.format(season)]=date_dummie_df[season]
#開催日数のダミー化
date_dummie_df=pd.get_dummies(test_date_df['range_date'])
ranges=[4,5,6,7]
for num_range in ranges:
    test_date_df['range_{}'.format(num_range)]=date_dummie_df[num_range]

#開催日数のうちの何日目かのダミー化
date_dummie_df=pd.get_dummies(test_date_df['num_date'])
num_dates=[1,2,3,4,5,6,7]
for num_date in num_dates:
    test_date_df['num_date_{}'.format(num_date)]=date_dummie_df[num_date]
test_date_df


c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

,date,money,number_race,racer_1_ID,racer_1_age,racer_1_ave_st,racer_1_bo,racer_1_doub,racer_1_male,racer_1_mo,...,range_5,range_6,range_7,num_date_1,num_date_2,num_date_3,num_date_4,num_date_5,num_date_6,num_date_7
15171,2019-04-01,5780.0,2,4501,31.0,0.16,19.0,0.453,0,72.0,...,0,1,0,1,0,0,0,0,0,0
15172,2019-04-01,790.0,3,3302,52.0,0.18,17.0,0.328,0,15.0,...,0,1,0,1,0,0,0,0,0,0
15173,2019-04-01,390.0,4,4050,38.0,0.16,13.0,0.525,0,68.0,...,0,1,0,1,0,0,0,0,0,0
15174,2019-04-01,3060.0,5,4373,31.0,0.19,51.0,0.344,0,44.0,...,0,1,0,1,0,0,0,0,0,0
15175,2019-04-01,1360.0,6,4419,32.0,0.20,63.0,0.245,0,50.0,...,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,2019-03-25,2190.0,8,3834,45.0,0.16,53.0,0.234,1,43.0,...,0,1,0,0,0,0,0,0,1,0
17286,2019-03-25,1170.0,9,4319,34.0,0.18,56.0,0.404,1,66.0,...,0,1,0,0,0,0,0,0,1,0
17287,2019-03-25,840.0,10,4543,32.0,0.19,48.0,0.398,1,14.0,...,0,1,0,0,0,0,0,0,1,0
17288,2019-03-25,1760.0,11,4448,32.0,0.16,57.0,0.483,1,21.0,...,0,1,0,0,0,0,0,0,1,0


In [11]:
test_date_df[850:900]

,date,money,number_race,racer_1_ID,racer_1_age,racer_1_ave_st,racer_1_bo,racer_1_doub,racer_1_male,racer_1_mo,...,month_7,month_8,month_9,month_10,month_11,month_12,season_sp,season_su,season_au,season_wi
16021,2019-08-31,3340.0,9,4950,26.0,0.15,31.0,0.252,1,37.0,...,0,1,0,0,0,0,0,1,0,0
16022,2019-08-31,790.0,10,4771,26.0,0.16,41.0,0.406,1,23.0,...,0,1,0,0,0,0,0,1,0,0
16023,2019-08-31,2890.0,11,4787,31.0,0.14,34.0,0.469,1,41.0,...,0,1,0,0,0,0,0,1,0,0
16024,2019-08-31,4610.0,12,4816,28.0,0.14,35.0,0.452,1,58.0,...,0,1,0,0,0,0,0,1,0,0
16025,2019-09-01,2690.0,1,4932,23.0,0.15,11.0,0.455,1,29.0,...,0,0,1,0,0,0,0,0,1,0
16026,2019-09-01,820.0,2,4852,25.0,0.15,16.0,0.429,1,64.0,...,0,0,1,0,0,0,0,0,1,0
16027,2019-09-01,2310.0,3,4908,24.0,0.13,23.0,0.500,1,57.0,...,0,0,1,0,0,0,0,0,1,0
16028,2019-09-01,5530.0,4,4762,29.0,0.13,25.0,0.514,1,53.0,...,0,0,1,0,0,0,0,0,1,0
16029,2019-09-01,1540.0,5,4786,31.0,0.16,6.0,0.355,1,16.0,...,0,0,1,0,0,0,0,0,1,0
16030,2019-09-01,1410.0,6,4750,32.0,0.16,47.0,0.243,1,46.0,...,0,0,1,0,0,0,0,0,1,0


### 学習データのベースを作成(validとtrainに分割)

In [22]:
#result_comと配当金を戻す
clustaring_df['result_com']=result_df['result_com']#正解ラベルを戻してあげる
clustaring_df['money']=result_df['money']#配当金を戻してあげる
df=clustaring_df

#学習、テストデータ切り分け
num_data = len(df) 
num_valid = int(num_data/6.0)
valid_df = df.loc[(num_data-num_valid):num_data, :]#なるべく最新のデータを未知データに
train_df = df.loc[0:(num_data-num_valid-1), :]
train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])


#### モデルの性能評価用のデータフレーム (モデルを使うかを判断する閾値を収納する)

In [23]:
#出現数の分布
result_com_s=valid_df['result_com'].value_counts()
result_com_s=result_com_s.sort_index()
gain_mean=valid_df.groupby('result_com')['money'].mean()
gain_mean=gain_mean.sort_index()

gain_median=valid_df.groupby('result_com')['money'].median()
gain_median=gain_median.sort_index()
result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                            'result_com_num':result_com_s.values,
                            'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                            'gain_mean':gain_mean.values,
                            'gain_median':gain_median.values,})
#result_com_df=result_com_df.set_index('result_com')

In [24]:
result_com_df

,result_com,result_com_num,result_com_per,gain_mean,gain_median
0,1.0,234,7.430930,1094.102564,785.0
1,2.0,204,6.478247,1280.196078,930.0
2,3.0,145,4.604636,2087.103448,1350.0
3,4.0,89,2.826294,2414.943820,1610.0
4,5.0,180,5.716100,1319.111111,950.0
...,...,...,...,...,...
112,115.0,2,0.063512,10300.000000,10300.0
113,116.0,2,0.063512,21955.000000,21955.0
114,117.0,3,0.095268,34193.333333,42250.0
115,118.0,3,0.095268,18846.666667,8670.0


In [25]:
valid_df[20:50]

,racer_1_age,racer_1_ave_st,racer_1_doub,racer_1_rank,racer_2_age,racer_2_ave_st,racer_2_doub,racer_2_rank,racer_3_age,racer_3_ave_st,...,racer_6_bo_94,racer_6_bo_95,racer_6_bo_96,racer_6_bo_97,racer_6_bo_98,racer_6_bo_99,num=8,num=10,result_com,money
15766,39.0,0.14,0.479,4,45.0,0.16,0.607,4,50.0,0.14,...,0,0,0,0,0,0,2,9,1.0,1100.0
15767,23.0,0.14,0.382,3,44.0,0.14,0.610,4,27.0,0.14,...,0,0,0,0,0,0,2,9,6.0,3470.0
15768,31.0,0.13,0.598,4,28.0,0.15,0.590,4,30.0,0.17,...,0,0,0,0,0,0,7,8,13.0,950.0
15769,45.0,0.13,0.468,4,23.0,0.12,0.564,4,34.0,0.14,...,0,0,0,0,0,0,0,7,64.0,5430.0
15770,37.0,0.13,0.544,4,23.0,0.13,0.370,3,33.0,0.16,...,0,0,0,0,0,0,7,8,10.0,690.0
15771,33.0,0.16,0.403,3,30.0,0.17,0.408,3,30.0,0.16,...,0,0,0,0,0,0,0,7,1.0,930.0
15772,39.0,0.14,0.479,4,35.0,0.15,0.458,4,26.0,0.15,...,0,0,0,0,0,0,0,7,16.0,3500.0
15773,30.0,0.18,0.378,3,32.0,0.20,0.461,4,50.0,0.15,...,0,0,0,0,0,0,6,2,48.0,18700.0
15774,38.0,0.15,0.426,4,28.0,0.14,0.400,3,44.0,0.14,...,0,0,0,0,0,0,6,9,4.0,4490.0
15775,48.0,0.17,0.442,4,50.0,0.14,0.443,4,31.0,0.16,...,0,0,0,0,0,0,3,6,14.0,5170.0


In [26]:
result_valid_df[20:50]

,racer_1_age,racer_1_ave_st,racer_1_doub,racer_1_rank,racer_2_age,racer_2_ave_st,racer_2_doub,racer_2_rank,racer_3_age,racer_3_ave_st,...,racer_6_bo_94,racer_6_bo_95,racer_6_bo_96,racer_6_bo_97,racer_6_bo_98,racer_6_bo_99,num=8,num=10,result_com,money
15766,39.0,0.14,0.479,4,45.0,0.16,0.607,4,50.0,0.14,...,0,0,0,0,0,0,1,0,0,1100.0
15767,23.0,0.14,0.382,3,44.0,0.14,0.610,4,27.0,0.14,...,0,0,0,0,0,0,1,7,0,3470.0
15768,31.0,0.13,0.598,4,28.0,0.15,0.590,4,30.0,0.17,...,0,0,0,0,0,0,4,5,0,950.0
15769,45.0,0.13,0.468,4,23.0,0.12,0.564,4,34.0,0.14,...,0,0,0,0,0,0,7,8,0,5430.0
15770,37.0,0.13,0.544,4,23.0,0.13,0.370,3,33.0,0.16,...,0,0,0,0,0,0,7,8,0,690.0
15771,33.0,0.16,0.403,3,30.0,0.17,0.408,3,30.0,0.16,...,0,0,0,0,0,0,7,8,0,930.0
15772,39.0,0.14,0.479,4,35.0,0.15,0.458,4,26.0,0.15,...,0,0,0,0,0,0,7,8,0,3500.0
15773,30.0,0.18,0.378,3,32.0,0.20,0.461,4,50.0,0.15,...,0,0,0,0,0,0,6,0,0,18700.0
15774,38.0,0.15,0.426,4,28.0,0.14,0.400,3,44.0,0.14,...,0,0,0,0,0,0,6,7,0,4490.0
15775,48.0,0.17,0.442,4,50.0,0.14,0.443,4,31.0,0.16,...,0,0,0,0,0,0,3,4,0,5170.0


## モデリング(探索的にパラメータを決める)

In [27]:
#result_com=1
#depth=5
model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
for result_com_number in result_com_df['result_com'].values:
    print(result_com_number)
    result_com=result_com_number
    #result_comごとの閾値の決定========================================================================
    
    gain_th=10#利益率の閾値
    result_s=result_com_df[result_com_df['result_com']==result_com]
    buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
    num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
    #===============================================================================
    #学習データのラベル変換==========================================================
    result_train_df=train_df.copy() 
    result_arr=[0]*len(result_train_df)
    i=0
    for result in result_train_df['result_com']:
        if ((result==result_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1
    result_train_df['result_com']=result_arr
    result_valid_df=valid_df.copy() 
    result_arr=[0]*len(result_valid_df)
    i=0
    for result in result_valid_df['result_com']:
        if ((result==result_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1

    result_valid_df['result_com']=result_arr

    result_train_df['money']=train_money
    result_valid_df['money']=valid_money
    #学習データラベル変換終わり============================================

    for_arr=np.arange(1,100,1)
    accuracy_arr=[0]*len(for_arr)
    target_per_arr=[0]*len(for_arr)
    pred_0=[0]*len(for_arr)
    gain_arr=[0]*len(for_arr)
    model_gain_arr=[0]*len(result_valid_df)
    valid_gain_arr=valid_money.values

    depths_arr=[4,5,6,7,8]
    for depth in depths_arr:
        for sum_target_per in tqdm(for_arr):
            
            index=sum_target_per-1
            target_per=50+sum_target_per
            target_per_arr[index]=target_per

            #モデルの評価指標値を格納するseries======================
            model_score_s=pd.Series(index=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=1)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1]
            len_1=len(target_1_df)
            target_0_df=target_df[target_df['result_com']==0]
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df])
            #以下学習パート======================================================
            target_x_train=target_train_df.drop('money',axis=1)
            target_x_train=target_x_train.drop('result_com',axis=1)
            target_y_train=target_train_df['result_com']
            #テストデータ
            target_y_valid=result_valid_df['result_com']
            target_x_valid=result_valid_df.drop('money',axis=1)
            target_x_valid=target_x_valid.drop('result_com',axis=1)
            RF = RandomForestClassifier(random_state=1,n_estimators=1000,max_depth=depth)
            RF = RF.fit(target_x_train,target_y_train)


            #以下精度検証(１の正答率のみ調査)

            # 未知データに対する予測値
            predict_y_valid = RF.predict(target_x_valid)

            #[1]の正答率を見る
            pred_valid_df=pd.DataFrame({'pred':predict_y_valid
                                      , 'valid':target_y_valid})
            num_1=len(pred_valid_df[pred_valid_df['valid']==1])
            count=0
            #追加　配当金の情報も考慮する。
            gain_index=0
            model_gain_arr=[0]*len(result_valid_df)
            for _, s in pred_valid_df.iterrows():
                if ((s['pred']==1) and (s['valid']==1)):
                    count+=1#的中回数
                    model_gain_arr[gain_index]=valid_gain_arr[gain_index]
                gain_index+=1
            #print('test accyracy: {}'.format((count/num_1)*100))
            gain_arr[index]=sum(model_gain_arr)
            accuracy_arr[index]=(count/num_1)*100
            try:
                pred_0[index]=pred_valid_df['pred'].value_counts()[0]
            except:
                pred_0[index]=0
            #scoreのseriesに情報書き込み==================
            model_score_s['総収益']=sum(model_gain_arr)
            model_score_s['投資金額']=100*sum(predict_y_valid)
            model_score_s['出現数']=sum(target_y_valid)
            model_score_s['購買予測数']=sum(predict_y_valid)
            model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
            model_score_s['購買的中率']=(count/sum(predict_y_valid))*100
            model_score_s['的中数']=count
            model_score_df=model_score_df.append(model_score_s,ignore_index=True)
            
#モデルの「スコアを保存        
model_score_df.to_csv('model_score.csv', encoding='utf_8_sig')
            
            

  0%|                                                                                                                                                                                                                | 0/99 [00:00<?, ?it/s]

1.0


 91%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                  | 90/99 [05:14<00:34,  3.81s/it]c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in double_scalars
c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in longlong_scalars
 93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉              | 92/99 [05:22<00:26,  3.83s/it]c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in double_scalars
c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value 

2.0


 93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉              | 92/99 [06:05<00:30,  4.32s/it]c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in double_scalars
c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in longlong_scalars
 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉          | 94/99 [06:14<00:21,  4.34s/it]c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in double_scalars
c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value 

...

  0%|                                                                                                                                                                                                                | 0/99 [00:00<?, ?it/s]

118.0


  0%|                                                                                                                                                                                                                | 0/99 [00:00<?, ?it/s]

119.0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [04:10<00:00,  2.53s/it]


## モデルのスコアを保存

In [28]:
model_score_df.to_csv('model_score.csv', encoding='utf_8_sig')

### モデリング用＿動作確認

In [29]:
"""
result_com=1
depth=5
model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])

#result_comごとの閾値の決定========================================================================
gain_th=10#利益率の閾値
result_s=result_com_df[result_com_df['result_com']==result_com]
buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
#===============================================================================
#学習データのラベル変換==========================================================
result_train_df=train_df
result_arr=[0]*len(result_train_df)
i=0
for result in result_train_df['result_com']:
    if ((result==result_com)):
        result_arr[i]=1
    else:
        result_arr[i]=0
    i+=1
result_train_df['result_com']=result_arr

result_valid_df=valid_df
result_arr=[0]*len(result_valid_df)
i=0
for result in result_valid_df['result_com']:
    if ((result==result_com)):
        result_arr[i]=1
    else:
        result_arr[i]=0
    i+=1

result_valid_df['result_com']=result_arr

result_train_df['money']=train_money
result_valid_df['money']=valid_money
#学習データラベル変換終わり============================================

for_arr=np.arange(1,100,1)
accuracy_arr=[0]*len(for_arr)
target_per_arr=[0]*len(for_arr)
pred_0=[0]*len(for_arr)
gain_arr=[0]*len(for_arr)
model_gain_arr=[0]*len(result_valid_df)
valid_gain_arr=valid_money.values
for sum_target_per in for_arr:
    
    index=sum_target_per-1
    target_per=50+sum_target_per
    target_per_arr[index]=target_per
    
    #モデルの評価指標値を格納するseries======================
    model_score_s=pd.Series(index=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
    model_score_s['target_com']=result_com#目標としているresult_comラベル番号
    model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
    model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
    #======================
    #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
    # 一層目の判別機のtrainデータ　:terget_result_df
    target_df=result_train_df#ベースのデータフレームをコピー
    target_df=target_df.sample(frac=1, random_state=1)#シャッフル、時系列の偏りを無くす
    target_1_df=target_df[target_df['result_com']==1]
    len_1=len(target_1_df)
    target_0_df=target_df[target_df['result_com']==0]
    len_0=len(target_0_df)
    target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
    target_train_df=pd.concat([target_1_df, target_0_df])
    #以下学習パート======================================================
    target_x_train=target_train_df.drop('money',axis=1)
    target_x_train=target_x_train.drop('result_com',axis=1)
    target_y_train=target_train_df['result_com']
    #テストデータ
    target_y_valid=result_valid_df['result_com']
    target_x_valid=result_valid_df.drop('money',axis=1)
    target_x_valid=target_x_valid.drop('result_com',axis=1)
    RF = RandomForestClassifier(random_state=1,n_estimators=1000,max_depth=depth)
    RF = RF.fit(target_x_train,target_y_train)


    #以下精度検証(１の正答率のみ調査)

    # 未知データに対する予測値
    predict_y_valid = RF.predict(target_x_valid)

    #[1]の正答率を見る
    pred_valid_df=pd.DataFrame({'pred':predict_y_valid
                              , 'valid':target_y_valid})
    num_1=len(pred_valid_df[pred_valid_df['valid']==1])
    count=0
    #追加　配当金の情報も考慮する。
    gain_index=0
    model_gain_arr=[0]*len(result_valid_df)
    for _, s in pred_valid_df.iterrows():
        if ((s['pred']==1) and (s['valid']==1)):
            count+=1#的中回数
            model_gain_arr[gain_index]=valid_gain_arr[gain_index]
        gain_index+=1
    #print('test accyracy: {}'.format((count/num_1)*100))
    gain_arr[index]=sum(model_gain_arr)
    accuracy_arr[index]=(count/num_1)*100
    try:
        pred_0[index]=pred_valid_df['pred'].value_counts()[0]
    except:
        pred_0[index]=0
    #scoreのseriesに情報書き込み==================
    model_score_s['総収益']=sum(model_gain_arr)
    model_score_s['投資金額']=100*sum(predict_y_valid)
    model_score_s['出現数']=sum(target_y_valid)
    model_score_s['購買予測数']=sum(predict_y_valid)
    model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
    model_score_s['購買的中率']=(count/sum(predict_y_valid))*100
    model_score_s['的中数']=count
    model_score_df=model_score_df.append(model_score_s,ignore_index=True)

"""

"\nresult_com=1\ndepth=5\nmodel_score_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])\n\n#result_comごとの閾値の決定========================================================================\ngain_th=10#利益率の閾値\nresult_s=result_com_df[result_com_df['result_com']==result_com]\nbuy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値\nnum_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)\n#===============================================================================\n#学習データのラベル変換==========================================================\nresult_train_df=train_df\nresult_arr=[0]*len(result_train_df)\ni=0\nfor result in result_train_df['result_com']:\n    if ((result==result_com)):\n        result_arr[i]=1\n    else:\n        result_arr[i]=0\n    i+=1\nresult_train_df['result_com']=result_arr\n\nresult_valid_df=valid_df\nresult_arr=[0]*len(result_valid_df)\ni=0\nfor result in result_valid_df['result_com']:

In [12]:
100*sum(predict_y_valid)

5900

In [13]:
(sum(model_gain_arr)/100*sum(predict_y_valid))

3262.7

In [54]:
model_score_df[:30]

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数
0,1.0,4.0,51.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
1,1.0,4.0,52.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
2,1.0,4.0,53.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
3,1.0,4.0,54.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
4,1.0,4.0,55.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
5,1.0,4.0,56.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
6,1.0,4.0,57.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
7,1.0,4.0,58.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
8,1.0,4.0,59.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
9,1.0,4.0,60.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0


In [20]:
result_s

,result_com,result_com_num,result_com_per,gain_mean,gain_median
0,1.0,234,7.43093,1094.102564,785.0
